In [1]:
#!pip install --upgrade pip

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import tifffile 
import os
import cv2 
import tensorflow as tf 
from tqdm import tqdm
import albumentations as A
import zipfile
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dense
#from keras.layers import Flatten
from keras.layers import AveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, Conv2DTranspose
from keras.layers.merge import concatenate #Concatenate (capital C) not working 
#from keras.utils.vis_utils import plot_model
from keras.layers import Dropout

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from tensorflow.keras.metrics import Recall, Precision
%matplotlib inline 

In [3]:
#elastic deformation with rotation and small shift for making the model robustness
def Augmentation(images,mask):
    def _transform(image,mask):
        transform=A.Compose([
         A.HorizontalFlip(p=0.25),
         A.VerticalFlip(p=0.25),
         A.RandomRotate90(p=.25),
         A.Transpose(p=0.25),
         A.ElasticTransform(p=.2, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    ])
        aug=transform(image=image,mask=mask)
        img=aug['image']
        img=tf.cast(img,tf.float64)
        msk=aug['mask']
        msk=tf.cast(msk,tf.float64)
        return img,msk
    image,mask=tf.numpy_function(_transform,[images,mask],[tf.float64,tf.float64])
    image.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    mask.set_shape([IMAGE_SIZE, IMAGE_SIZE, 1])
    
    return image,mask
from glob import glob
IMAGE_SIZE=512
AUTO = tf.data.experimental.AUTOTUNE
def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, "train/*")))
    print(len(images))
    masks = sorted(glob(os.path.join(path, "masks/*")))
    print(len(masks))
    total_size = len(images)
    valid_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y)
(train_x,train_y),(valid_x,valid_y)=load_data('../input/hubmap-1024x1024')
train_size=len(train_x)
valid_size=len(valid_x)
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x,(512,512),interpolation=cv2.INTER_AREA)
    x = x/255.0
    x=tf.cast(x,dtype=tf.float64)
    return x
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x,(512,512),interpolation=cv2.INTER_NEAREST)
    x = np.expand_dims(x, axis=-1)
    x=tf.cast(x,dtype=tf.float64)
    return x
def parser(x,y):
    def _parse(x,y):
        x=read_image(x)
        y=read_mask(y)
        return x,y
    x,y = tf.numpy_function(_parse, [x,y], [tf.float64,tf.float64])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 1])
    return x,y
    
def tf_dataset(x, y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(parser)
    dataset = dataset.map(Augmentation)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch)

    return dataset
#train_dataset=tf_dataset(train_x,train_y,batch=4)
#valid_dataset=tf_dataset(valid_x,valid_y,batch=4)

3849
3849


In [4]:
print(train_dataset)
print(valid_dataset)

NameError: name 'train_dataset' is not defined

In [5]:
IMAGE_WIDTH=1024
IMAGE_HEIGHT=1024
IMAGE_CHANNELS=3

# loss functions 


In [6]:
from tensorflow.keras.losses import binary_crossentropy
smooth = 1
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def binary_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred)

def binary_dice_loss(y_true, y_pred):
    d_loss= dice_loss(y_true, y_pred)
    b_loss= binary_loss(y_true, y_pred)
    loss=(d_loss+b_loss)
    return loss

def Tversky_Index(y_true,y_pred,smooth):
    y_true= tf.keras.layers.Flatten()(y_true)
    y_pred= tf.keras.layers.Flatten()(y_pred)
    true_positive= tf.reduce_sum(y_true*y_pred)
    false_negative=tf.reduce_sum(y_true*(1-y_pred))
    false_positive=tf.reduce_sum((1-y_true)*y_pred)
    alpha=.7
    TI=(true_positive+smooth)/(true_positive+alpha*false_negative+(1-alpha)*false_positive+smooth)
    return TI

def Tversky_loss(y_true,y_pred):
    return 1-Tversky_Index(y_true,y_pred,smooth=1)
def focal_Tversky_loss(y_true,y_pred):
    pt_1 = Tversky_Index(y_true, y_pred,smooth=1)
    gamma = 0.75
    return tf.math.pow((1-pt_1), gamma)




#model

In [7]:

from keras.applications import VGG19
from keras.layers import GlobalAveragePooling2D 
from keras.layers import multiply, Reshape
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.models import Model
from tensorflow.keras.applications import *

In [8]:
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = multiply([init, se])
    return x

def conv_block(inputs, filters):
    x = inputs

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = []

    model = VGG19(include_top=False, weights='../input/pretrained-vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', input_tensor=inputs)
    print(model.summary())
    names = ["block1_conv2", "block2_conv2", "block3_conv4", "block4_conv4"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("block5_conv4").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [256, 128, 64, 32]
    skip_connections.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = concatenate([x, skip_connections[i]])
        x = conv_block(x, f)

    return x
def encoder2(inputs):
    num_filters = [32, 64, 128, 256]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPooling2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 32]
    skip_2.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = concatenate([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, (1, 1), padding="same")(inputs)
    x = Activation('sigmoid')(x)
    return x

def Upsample(tensor, size):
    """Bilinear upsampling"""
    def _upsample(x, size):
        return tf.image.resize(images=x, size=size)
    return Lambda(lambda x: _upsample(x, size), output_shape=size)(tensor)
def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = concatenate([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y
def build_model(shape):
    inputs = Input(shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    outputs1 = output_block(x)

    x = inputs * outputs1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    outputs2 = output_block(x)
    #outputs = concatenate([outputs1, outputs2])
    #output=Conv2D(1,(2,2),activation="sigmoid",padding="same")(outputs)
    model = Model(inputs, outputs2)
    return model


In [9]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)

In [10]:
import tensorflow as tf
desnet_model=tf.keras.models.load_model('../input/desnetpretrained6/models4.hd5',custom_objects={
        'focal_Tversky_loss': focal_Tversky_loss ,'dice_coef': dice_coef})


In [11]:
import rasterio 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import pathlib
import gc
from rasterio.windows import Window
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
WINDOW,MIN_OVERLAP=1024,300
p = pathlib.Path('../input/hubmap-kidney-segmentation')
NEW_SIZE=512
subm = {}
THRESHOLD=0.7
for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
                        total = len(list(p.glob('test/*.tiff')))):
    
    print(f'{i+1} Predicting {filename.stem}')
    
    dataset = rasterio.open(filename.as_posix(), transform = identity)
    slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
    preds = np.zeros(dataset.shape, dtype=np.uint8)
    
    for (x1,x2,y1,y2) in slices:
        image = dataset.read([1,2,3],
                    window=Window.from_slices((x1,x2),(y1,y2)))
        image = np.moveaxis(image, 0, -1)
        image = cv2.resize(image, (NEW_SIZE, NEW_SIZE),interpolation = cv2.INTER_AREA)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = image/255.0
        image = np.expand_dims(image, 0)
        
        pred = None
        
        
        if pred is None:
            pred = np.squeeze(desnet_model.predict(image))
        else:
            pred += np.squeeze(desnet_model.predict(image))
        
        pred = pred
        
        pred = cv2.resize(pred, (WINDOW, WINDOW))
        preds[x1:x2,y1:y2] += (pred > THRESHOLD).astype(np.uint8)
    
    preds = (preds > 0.7).astype(np.uint8)
    
    subm[i] = {'id':filename.stem, 'predicted': rle_encode_less_memory(preds)}
    del preds
    gc.collect()

  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/rasterio/__init__.py:221: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


1 Predicting afa5e8098


 20%|██        | 1/5 [06:51<27:27, 411.89s/it]

2 Predicting b9a3865fc


 40%|████      | 2/5 [12:30<19:29, 390.00s/it]

...

 60%|██████    | 3/5 [18:50<12:54, 387.02s/it]

4 Predicting b2dc8411c


 80%|████████  | 4/5 [20:48<05:06, 306.16s/it]

5 Predicting 26dc41664


100%|██████████| 5/5 [28:07<00:00, 337.57s/it]


In [12]:
submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv('submission.csv', index=False)
submission

,id,predicted
0,afa5e8098,15967613 19 16004411 29 16041210 34 16078009 3...
1,b9a3865fc,61146700 28 61177980 46 61209262 65 61240553 7...
2,c68fe75ea,21122582 19 21149419 28 21176255 41 21203093 4...
3,b2dc8411c,18270166 16 18284999 35 18285402 54 18285459 3...
4,26dc41664,24302898 31 24303173 14 24341056 54 24341148 2...


In [13]:
import tifffile 
p= pathlib.Path('../input/hubmap-kidney-segmentation')
import os 
from glob import glob 
path="../input/hubmap-kidney-segmentation/test"
data=sorted(glob(os.path.join(path,'test/*.tiff')))
print(data)

[]
